# SQL

In [1]:
import pandas as pd
import sqlalchemy

## Connect to database

In [2]:
# Connect to the movie db
engine = sqlalchemy.create_engine(
    "postgresql+psycopg2://", 
    connect_args={"database": "postgres", "user": "faculty", "host": "/var/run/postgresql"}
)
con = engine.connect()
con.execute("SET schema 'input'")

## Create tables
Create simple `customers` table from slides:

In [3]:
con.execute('''
    DROP TABLE IF EXISTS customers CASCADE;
    DROP SEQUENCE IF EXISTS customers_id_seq;
''')

In [4]:
con.execute("CREATE SEQUENCE customers_id_seq")
con.execute("""
CREATE TABLE customers (
    id BIGINT DEFAULT NEXTVAL('customers_id_seq') PRIMARY KEY,
    name TEXT NOT NULL,
    address TEXT
)
""")

In [5]:
con.execute("""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname = 'input'
""")

Create `product` table from slides:

In [6]:
con.execute('''
    DROP TABLE IF EXISTS products CASCADE;
    DROP SEQUENCE IF EXISTS products_id_seq;
''')

In [7]:
con.execute("CREATE SEQUENCE products_id_seq")
con.execute("""
CREATE TABLE products (
    id BIGINT DEFAULT NEXTVAL('products_id_seq') PRIMARY KEY,
    name TEXT NOT NULL
)
""")

Create `sales` table from slides:

In [8]:
con.execute('''
    DROP TABLE IF EXISTS sales CASCADE;
    DROP SEQUENCE IF EXISTS sales_id_seq;
''')

In [9]:
con.execute("CREATE SEQUENCE sales_id_seq")
con.execute("""
CREATE TABLE sales (
    id BIGINT DEFAULT NEXTVAL('sales_id_seq') PRIMARY KEY,
    customer_id BIGINT,
    product_id BIGINT,
    quantity BIGINT,
    FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE,
    FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE
)
""")

# Insert values

In [10]:
con.execute("INSERT INTO customers (name, address) VALUES ('Andy Brookes', 'London')")
con.execute("INSERT INTO customers (name, address) VALUES ('Jake Dunn', 'London')")
con.execute("INSERT INTO customers (name, address) VALUES ('Bill Gates', 'Seattle')")
con.execute("INSERT INTO customers (name, address) VALUES ('Steve Jobs', 'Palo Alto')")
con.execute("INSERT INTO customers (name, address) VALUES ('Random guy', NULL)")

In [11]:
con.execute("INSERT INTO products (name) VALUES ('iPhone')")
con.execute("INSERT INTO products (name) VALUES ('Xbox')")
con.execute("INSERT INTO products (name) VALUES ('PlayStation')")
con.execute("INSERT INTO products (name) VALUES ('Wii')")

In [12]:
con.execute("""
    INSERT INTO sales (customer_id, product_id, quantity)
    VALUES (1, 1, 1), (2, 2, 3), (3, 1, 2), (5, 3, 10)
""")

## Select results

The full `customers` table:

In [13]:
q = """
SELECT * 
FROM customers 
"""
df = pd.read_sql(q, con)
df

,id,name,address
0,1,Andy Brookes,London
1,2,Jake Dunn,London
2,3,Bill Gates,Seattle
3,4,Steve Jobs,Palo Alto
4,5,Random guy,None


The full `products` table:

In [14]:
q = """
SELECT * 
FROM products 
"""
df = pd.read_sql(q, con)
df

,id,name
0,1,iPhone
1,2,Xbox
2,3,PlayStation
3,4,Wii


The full `sales` table:

In [15]:
q = """
SELECT * 
FROM sales 
"""
df = pd.read_sql(q, con)
df

,id,customer_id,product_id,quantity
0,1,1,1,1
1,2,2,2,3
2,3,3,1,2
3,4,5,3,10


Perform a projection, i.e. taking vertical slices of a table, and only recording values once:

In [16]:
q = """
SELECT DISTINCT address 
FROM customers 
"""
df = pd.read_sql(q, con)
df

,address
0,None
1,Seattle
2,London
3,Palo Alto


Perform a selection, i.e. filter rows of a table based on some predicate you are interested in:

In [17]:
q = """
SELECT name 
FROM customers 
WHERE address = 'London'
"""
df = pd.read_sql(q, con)
df

,name
0,Andy Brookes
1,Jake Dunn


You can order your results:

In [18]:
q = """
SELECT name 
FROM customers 
ORDER BY name ASC
"""
df = pd.read_sql(q, con)
df

,name
0,Andy Brookes
1,Bill Gates
2,Jake Dunn
3,Random guy
4,Steve Jobs


You can also aggregate results:

In [19]:
q = """
SELECT product_id, SUM(quantity) 
FROM sales 
GROUP BY product_id
"""
df = pd.read_sql(q, con)
df

,product_id,sum
0,3,10.0
1,2,3.0
2,1,3.0


In [20]:
q = """
SELECT product_id, SUM(quantity) AS orders
FROM sales 
GROUP BY product_id
"""
df = pd.read_sql(q, con)
df

,product_id,orders
0,3,10.0
1,2,3.0
2,1,3.0


Using `IN` in predicates, to check membership to a collection:

In [21]:
q = """
SELECT name 
FROM customers 
WHERE address IN ('Palo Alto', 'Seattle')
"""
df = pd.read_sql(q, con)
df

,name
0,Bill Gates
1,Steve Jobs


And of course if we negate...

In [22]:
q = """
SELECT name 
FROM customers 
WHERE address NOT IN ('Palo Alto', 'Seattle')
"""
df = pd.read_sql(q, con)
df

,name
0,Andy Brookes
1,Jake Dunn


WÄT???? :|

What happened to `Random guy`?

NULL in SQL can be confusing. This is because you shouldn't think of them as a `None`, but rather as an `Unknown` value. As a result, e.g., `NULL IN <anything>` is `NULL`.

When writing predicates involving NULL remember not to use `=` but `IS`, this will make your life easier:

In [23]:
q = """
SELECT name 
FROM customers 
WHERE address IS NOT NULL
"""
df = pd.read_sql(q, con)
df

,name
0,Andy Brookes
1,Jake Dunn
2,Bill Gates
3,Steve Jobs


Set operations: `INTERSECT` and `UNION`

In [24]:
q = """
(SELECT name, address 
FROM customers
WHERE address IS NOT NULL)
INTERSECT
(SELECT name, address
FROM customers
WHERE id > 1)
"""
df = pd.read_sql(q, con)
df

,name,address
0,Bill Gates,Seattle
1,Steve Jobs,Palo Alto
2,Jake Dunn,London


In [25]:
q = """
(SELECT name, address 
FROM customers
WHERE address IS NOT NULL)
UNION
(SELECT name, address
FROM customers
WHERE id > 1)
"""
df = pd.read_sql(q, con)
df

,name,address
0,Bill Gates,Seattle
1,Steve Jobs,Palo Alto
2,Jake Dunn,London
3,Random guy,None
4,Andy Brookes,London


Aggregating results from multiple tables with JOIN. 

In [26]:
q = """
SELECT products.name, sales.quantity 
FROM products
JOIN sales
ON products.id = sales.product_id
"""
df = pd.read_sql(q, con)
df

,name,quantity
0,iPhone,1
1,Xbox,3
2,iPhone,2
3,PlayStation,10


Example of LEFT JOIN

In [27]:
q = """
SELECT products.name, sales.quantity 
FROM products
LEFT OUTER JOIN sales
ON products.id = sales.product_id
"""
df = pd.read_sql(q, con)
df

,name,quantity
0,iPhone,1.0
1,Xbox,3.0
2,iPhone,2.0
3,PlayStation,10.0
4,Wii,NaN


Example of less common `JOIN`:

In [28]:
q = """
SELECT products.name, sales.quantity 
FROM products
FULL OUTER JOIN sales
ON products.id = sales.product_id
WHERE products.id IS NULL OR sales.id IS NULL
"""
df = pd.read_sql(q, con)
df

,name,quantity
0,Wii,None


## Update values

Update statements can grow more complicated, but here is a taster:

In [29]:
con.execute("UPDATE customers SET name = 'Andrew Brookes' WHERE name = 'Andy Brookes'")
q = """
SELECT *
FROM customers
"""
df = pd.read_sql(q, con)
df

,id,name,address
0,2,Jake Dunn,London
1,3,Bill Gates,Seattle
2,4,Steve Jobs,Palo Alto
3,5,Random guy,None
4,1,Andrew Brookes,London


## Delete entries

A simple example of how to delete rows matching a filer:

In [30]:
con.execute("DELETE FROM customers WHERE name = 'Andrew Brookes'")
q = """
SELECT *
FROM customers
"""
df = pd.read_sql(q, con)
df

,id,name,address
0,2,Jake Dunn,London
1,3,Bill Gates,Seattle
2,4,Steve Jobs,Palo Alto
3,5,Random guy,None


## Drop tables -- BE CAREFUL!!! :@

Deleting tables is not something you will have to do too often, so when doing this it is worth being extra careful. You do NOT want to be dropping tables late on a Friday night, especially if you want to avoid explaining a client/manager why you lost all your data :)

Let's remove the tables we have created so far, to leave us with a clean state for the exercise:

In [31]:
con.execute("DROP TABLE sales")
con.execute("DROP SEQUENCE sales_id_seq")

con.execute("DROP TABLE products")
con.execute("DROP SEQUENCE products_id_seq")

con.execute("DROP TABLE customers")
con.execute("DROP SEQUENCE customers_id_seq")